In [1]:
from pykeen.pipeline import pipeline
from pykeen.datasets import Nations, get_dataset
import torch
from pykeen.evaluation import evaluate, RankBasedEvaluator
from pykeen.metrics.ranking import HitsAtK
import pandas as pd


import logging
from pathlib import Path

import click
import more_click
import torch
from pykeen.evaluation import RankBasedEvaluator
from pykeen.losses import NSSALoss,CrossEntropyLoss
from pykeen.models.inductive import InductiveNodePiece, InductiveNodePieceGNN
from pykeen.trackers import ConsoleResultTracker, WANDBResultTracker, FileResultTracker
from pykeen.training import SLCWATrainingLoop
from pykeen.typing import TESTING, TRAINING, VALIDATION
from pykeen.utils import resolve_device, set_random_seed
from torch.optim import Adam


from pykeen.metrics.ranking import HitsAtK

from pathlib import Path

from pykeen.datasets.inductive.base import DisjointInductivePathDataset
from pykeen.datasets.base import PathDataset, Dataset
from typing_extensions import Literal
import os
from pykeen.hpo import hpo_pipeline
from pykeen.triples import TriplesFactory
from pykeen.models import InductiveNodePiece, TransE, RGCN, QuatE, RotatE, TuckER
from pykeen.typing import TESTING, TRAINING, VALIDATION

import time

import platform

import sys

import cpuinfo

import psutil

import subprocess

import zipfile

seed = 1234

In [2]:
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:512"

In [3]:
def show_metrics(dictionary,model_name,csv_name):
    for key in dictionary.keys():
        print(key)
        df = pd.DataFrame(dictionary[key])
        df.to_csv(f"{model_name}/{model_name}_{csv_name}_{key}.csv")
        print(df)

In [4]:
DATA_TYPE = "_transductive.tsv"
TRAIN_PATH = "MSCallGraph_0_train" + DATA_TYPE
TEST_PATH = "MSCallGraph_0_test" + DATA_TYPE
VALIDATE_PATH = "MSCallGraph_0_validation" + DATA_TYPE

In [5]:
dataset = PathDataset(training_path = TRAIN_PATH,
                     testing_path = TEST_PATH,
                     validation_path = VALIDATE_PATH,
                      eager = True
                     )


In [6]:
model_name = 'tuckER_0_transductive'

In [7]:
tracker = ConsoleResultTracker()

In [8]:
loss = NSSALoss() #used by RotatE and NodePiece
embedding_dim = 200
batch_size = 512

In [9]:
model = TuckER(
        triples_factory=dataset.training,
        random_seed = seed,
        loss = loss,
        embedding_dim = embedding_dim
    ).to(resolve_device())
print(f"Number of parameters: {sum(p.numel() for p in model.parameters())}")
print(f"Space occupied: {model.num_parameter_bytes} bytes")

Number of parameters: 9479200
Space occupied: 37916800 bytes


In [10]:
directory = model_name

if not os.path.exists(directory):
    os.makedirs(directory)
    print(f'Directory {directory} created successfully!')
else:
    print(f'Directory {directory} already exists.')

Directory tuckER_0_transductive already exists.


In [11]:
learning_rate = 1e-2 #1e-3 and 1e-4 stops after 20 iterations and actually does not reduce the error too much
optimizer = Adam(params=model.parameters(), lr=learning_rate)
num_epochs = 2000
patience = 20

In [12]:
metrics = ['meanreciprocalrank', HitsAtK(1),
                 HitsAtK(3), HitsAtK(5), HitsAtK(10)]

train_evaluator = RankBasedEvaluator(
        metrics=metrics,
        add_defaults=False,
    )
valid_evaluator = RankBasedEvaluator(
        metrics=metrics,
        add_defaults=False,
    )
test_evaluator = RankBasedEvaluator(
        metrics = metrics,
        add_defaults=False
    )

In [13]:
from pykeen.stoppers import EarlyStopper

stopper = EarlyStopper(
    model = model,
    metric='meanreciprocalrank',
    patience=patience,
    frequency=1,
    evaluator = valid_evaluator,
    training_triples_factory = dataset.training,
    evaluation_triples_factory = dataset.validation,
    result_tracker = tracker

)



In [14]:
# default training regime is negative sampling (SLCWA)
# you can also use the 1-N regime with the LCWATrainingLoop
# the LCWA loop does not need negative sampling kwargs, but accepts label_smoothing in the .train() method
training_loop = SLCWATrainingLoop(
        triples_factory=dataset.training,
        model=model,
        result_tracker=tracker,
        optimizer=optimizer
)

In [15]:
training_start = time.time()
train_epoch =  training_loop.train(
        triples_factory=dataset.training,
        num_epochs=num_epochs,
        stopper = stopper,
        batch_size = batch_size
        
    )
training_duration = time.time() - training_start

Training epochs on cuda:0:   0%|          | 0/2000 [00:00<?, ?epoch/s]

Training batches on cuda:0:   0%|          | 0/20 [00:00<?, ?batch/s]

Step: 1
Metric: loss = 4.49360089302063
Step: 1
Metric: validation.head.optimistic.inverse_harmonic_mean_rank = 0.002372526564185936
Metric: validation.tail.optimistic.inverse_harmonic_mean_rank = 0.00264681493226323
Metric: validation.both.optimistic.inverse_harmonic_mean_rank = 0.0025096707482245833
Metric: validation.head.realistic.inverse_harmonic_mean_rank = 0.002372526563704014
Metric: validation.tail.realistic.inverse_harmonic_mean_rank = 0.0026468150317668915
Metric: validation.both.realistic.inverse_harmonic_mean_rank = 0.0025096707977354527
Metric: validation.head.pessimistic.inverse_harmonic_mean_rank = 0.002372526564185936
Metric: validation.tail.pessimistic.inverse_harmonic_mean_rank = 0.00264681493226323
Metric: validation.both.pessimistic.inverse_harmonic_mean_rank = 0.0025096707482245833
Metric: validation.head.optimistic.hits_at_1 = 0.0003891050583657588
Metric: validation.tail.optimistic.hits_at_1 = 0.0011673151750972762
Metric: validation.both.optimistic.hits_at_1 = 

Training batches on cuda:0:   0%|          | 0/20 [00:00<?, ?batch/s]

Step: 2
Metric: loss = 4.3200380325317385
Step: 2
Metric: validation.head.optimistic.inverse_harmonic_mean_rank = 0.0036294408147907035
Metric: validation.tail.optimistic.inverse_harmonic_mean_rank = 0.0007880065118307648
Metric: validation.both.optimistic.inverse_harmonic_mean_rank = 0.0022087236633107346
Metric: validation.head.realistic.inverse_harmonic_mean_rank = 0.0036294409073889256
Metric: validation.tail.realistic.inverse_harmonic_mean_rank = 0.0007880065240897238
Metric: validation.both.realistic.inverse_harmonic_mean_rank = 0.002208723919466138
Metric: validation.head.pessimistic.inverse_harmonic_mean_rank = 0.0036294408147907035
Metric: validation.tail.pessimistic.inverse_harmonic_mean_rank = 0.00078800650374379
Metric: validation.both.pessimistic.inverse_harmonic_mean_rank = 0.002208723659267247
Metric: validation.head.optimistic.hits_at_1 = 0.0011673151750972762
Metric: validation.tail.optimistic.hits_at_1 = 0.0
Metric: validation.both.optimistic.hits_at_1 = 0.00058365758

...

Step: 156
Metric: loss = 0.05304127149283886
Step: 156
Metric: validation.head.optimistic.inverse_harmonic_mean_rank = 0.09570372398591402
Metric: validation.tail.optimistic.inverse_harmonic_mean_rank = 0.20248095226440818
Metric: validation.both.optimistic.inverse_harmonic_mean_rank = 0.14909233812516107
Metric: validation.head.realistic.inverse_harmonic_mean_rank = 0.09570372104644777
Metric: validation.tail.realistic.inverse_harmonic_mean_rank = 0.20248097181320193
Metric: validation.both.realistic.inverse_harmonic_mean_rank = 0.14909233152866364
Metric: validation.head.pessimistic.inverse_harmonic_mean_rank = 0.09570372390777407
Metric: validation.tail.pessimistic.inverse_harmonic_mean_rank = 0.20248095226440818
Metric: validation.both.pessimistic.inverse_harmonic_mean_rank = 0.1490923380860911
Metric: validation.head.optimistic.hits_at_1 = 0.04085603112840467
Metric: validation.tail.optimistic.hits_at_1 = 0.11284046692607004
Metric: validation.both.optimistic.hits_at_1 = 0.0768482

Training batches on cuda:0:   0%|          | 0/20 [00:00<?, ?batch/s]

Step: 157
Metric: loss = 0.05170220844447613
Step: 157
Metric: validation.head.optimistic.inverse_harmonic_mean_rank = 0.09598998373829065
Metric: validation.tail.optimistic.inverse_harmonic_mean_rank = 0.18033145849332816
Metric: validation.both.optimistic.inverse_harmonic_mean_rank = 0.1381607211158094
Metric: validation.head.realistic.inverse_harmonic_mean_rank = 0.09598998725414278
Metric: validation.tail.realistic.inverse_harmonic_mean_rank = 0.18033145368099213
Metric: validation.both.realistic.inverse_harmonic_mean_rank = 0.13816072046756744
Metric: validation.head.pessimistic.inverse_harmonic_mean_rank = 0.09598998371385566
Metric: validation.tail.pessimistic.inverse_harmonic_mean_rank = 0.18033145848006554
Metric: validation.both.pessimistic.inverse_harmonic_mean_rank = 0.1381607210969606
Metric: validation.head.optimistic.hits_at_1 = 0.04202334630350195
Metric: validation.tail.optimistic.hits_at_1 = 0.09494163424124513
Metric: validation.both.optimistic.hits_at_1 = 0.06848249

Training batches on cuda:0:   0%|          | 0/20 [00:00<?, ?batch/s]

Step: 158
Metric: loss = 0.05096820704638958
Step: 158
Metric: validation.head.optimistic.inverse_harmonic_mean_rank = 0.09585640858351532
Metric: validation.tail.optimistic.inverse_harmonic_mean_rank = 0.192244102044059
Metric: validation.both.optimistic.inverse_harmonic_mean_rank = 0.14405025531378715
Metric: validation.head.realistic.inverse_harmonic_mean_rank = 0.09585640579462051
Metric: validation.tail.realistic.inverse_harmonic_mean_rank = 0.19224409759044647
Metric: validation.both.realistic.inverse_harmonic_mean_rank = 0.1440502554178238
Metric: validation.head.pessimistic.inverse_harmonic_mean_rank = 0.09585640858351532
Metric: validation.tail.pessimistic.inverse_harmonic_mean_rank = 0.192244102044059
Metric: validation.both.pessimistic.inverse_harmonic_mean_rank = 0.14405025531378715
Metric: validation.head.optimistic.hits_at_1 = 0.041634241245136185
Metric: validation.tail.optimistic.hits_at_1 = 0.10739299610894941
Metric: validation.both.optimistic.hits_at_1 = 0.0745136186

In [16]:
torch.save(model,f"{model_name}/model.pth")
model = torch.load(f"{model_name}/model.pth")

In [17]:
print("Train error per epoch:")
df = pd.DataFrame(train_epoch)
print(df)
df.to_csv(f"{model_name}/{model_name}_train_error_per_epoch.csv")

Train error per epoch:
            0
0    4.493601
1    4.320038
2    3.832066
3    3.271578
4    2.660165
..        ...
153  0.050423
154  0.059477
155  0.053041
156  0.051702
157  0.050968

[158 rows x 1 columns]


In [18]:
training_evaluation_start = time.time()
# train
print("Train error")
show_metrics(train_evaluator.evaluate(
        model=model,
        mapped_triples=dataset.training.mapped_triples,
        additional_filter_triples=[
        dataset.training.mapped_triples,
    ]
    ).to_dict(),model_name,'train_metrics')
training_evaluation_duration = time.time() - training_evaluation_start

Train error


Evaluating on cuda:0:   0%|          | 0.00/10.4k [00:00<?, ?triple/s]

head
                            optimistic  realistic  pessimistic
inverse_harmonic_mean_rank    0.787964   0.787964     0.787964
hits_at_1                     0.707254   0.707254     0.707254
hits_at_3                     0.840767   0.840767     0.840767
hits_at_5                     0.883634   0.883634     0.883634
hits_at_10                    0.932280   0.932280     0.932280
tail
                            optimistic  realistic  pessimistic
inverse_harmonic_mean_rank    0.868546   0.868546     0.868546
hits_at_1                     0.816010   0.816010     0.816010
hits_at_3                     0.904055   0.904055     0.904055
hits_at_5                     0.932473   0.932473     0.932473
hits_at_10                    0.960986   0.960986     0.960986
both
                            optimistic  realistic  pessimistic
inverse_harmonic_mean_rank    0.828255   0.828255     0.828255
hits_at_1                     0.761632   0.761632     0.761632
hits_at_3                     0.872411  

In [19]:
validation_evaluation_start = time.time()
# validation
print("Validation error")
show_metrics(valid_evaluator.evaluate(
        model=model,
        mapped_triples=dataset.validation.mapped_triples,
        additional_filter_triples=[
            # filtering of other positive triples
            dataset.training.mapped_triples
        ],
    ).to_dict(),model_name,'validation_metrics')
validation_evaluation_duration = time.time() - validation_evaluation_start

Validation error


Evaluating on cuda:0:   0%|          | 0.00/2.57k [00:00<?, ?triple/s]

head
                            optimistic  realistic  pessimistic
inverse_harmonic_mean_rank    0.092816   0.092816     0.092816
hits_at_1                     0.038521   0.038521     0.038521
hits_at_3                     0.091051   0.091051     0.091051
hits_at_5                     0.129572   0.129572     0.129572
hits_at_10                    0.198833   0.198833     0.198833
tail
                            optimistic  realistic  pessimistic
inverse_harmonic_mean_rank    0.230014   0.230014     0.230014
hits_at_1                     0.151751   0.151751     0.151751
hits_at_3                     0.264981   0.264981     0.264981
hits_at_5                     0.309728   0.309728     0.309728
hits_at_10                    0.365759   0.365759     0.365759
both
                            optimistic  realistic  pessimistic
inverse_harmonic_mean_rank    0.161415   0.161415     0.161415
hits_at_1                     0.095136   0.095136     0.095136
hits_at_3                     0.178016  

In [20]:
testing_evaluation_start = time.time()
# result on the test set
print("Test error")
show_metrics(test_evaluator.evaluate(
        model=model,
        mapped_triples=dataset.testing.mapped_triples,
        additional_filter_triples=[
            # filtering of other positive triples
            dataset.training.mapped_triples,
            dataset.validation.mapped_triples,
        ],
    ).to_dict(),model_name,'test_metrics')
testing_evaluation_duration = time.time() - testing_evaluation_start

Test error


Evaluating on cuda:0:   0%|          | 0.00/3.27k [00:00<?, ?triple/s]

head
                            optimistic  realistic  pessimistic
inverse_harmonic_mean_rank    0.131150   0.131150     0.131150
hits_at_1                     0.058410   0.058410     0.058410
hits_at_3                     0.153517   0.153517     0.153517
hits_at_5                     0.207951   0.207951     0.207951
hits_at_10                    0.277982   0.277982     0.277982
tail
                            optimistic  realistic  pessimistic
inverse_harmonic_mean_rank    0.241126   0.241126     0.241126
hits_at_1                     0.168502   0.168502     0.168502
hits_at_3                     0.273700   0.273700     0.273700
hits_at_5                     0.313456   0.313456     0.313456
hits_at_10                    0.364526   0.364526     0.364526
both
                            optimistic  realistic  pessimistic
inverse_harmonic_mean_rank    0.186138   0.186138     0.186138
hits_at_1                     0.113456   0.113456     0.113456
hits_at_3                     0.213609  

In [21]:
infodict = {}
infodict['device'] = model.device
infodict['parameters bytes'] = model.num_parameter_bytes
infodict['number parameters'] = model.num_parameters
infodict['training duration'] = training_duration
infodict['training evaluation duration'] = training_evaluation_duration
infodict['validation evaluation duration'] = validation_evaluation_duration
infodict['testing evaluation duration'] = testing_evaluation_duration
infodict["Operating system name"] = platform.system()
infodict["Operating system version"] = platform.release()
infodict["Processor architecture"] = platform.machine()
infodict["Python version"] = sys.version
infodict["Processor model name"] = cpuinfo.get_cpu_info()['brand_raw']
infodict['Number cpu cores'] = os.cpu_count()
infodict["Total physical memory"] = psutil.virtual_memory().total

In [22]:

output = subprocess.check_output(['nvidia-smi', '--query-gpu=name', '--format=csv'])
output = output.decode('utf-8')  # convert byte string to regular string

# split output into rows and remove header row
rows = output.strip().split('\n')[1:]

# extract GPU names from each row
gpu_names = []
for row in rows:
    name = row.strip()
    gpu_names.append(name)

infodict['GPU'] = gpu_names[0]


In [23]:
infodict['loss'] = NSSALoss
infodict['embedding_dim'] = embedding_dim
infodict['learning_rate'] = learning_rate
infodict['optimizer'] = Adam
infodict['num_epochs'] = num_epochs
infodict['patience'] = patience
infodict['batch_size'] = batch_size

In [24]:
info_df = pd.DataFrame(columns=['name','value'], data = infodict.items())
info_df.to_csv(f"{model_name}/{model_name}_information.csv")
print(info_df)

                              name  \
0                           device   
1                 parameters bytes   
2                number parameters   
3                training duration   
4     training evaluation duration   
5   validation evaluation duration   
6      testing evaluation duration   
7            Operating system name   
8         Operating system version   
9           Processor architecture   
10                  Python version   
11            Processor model name   
12                Number cpu cores   
13           Total physical memory   
14                             GPU   
15                            loss   
16                   embedding_dim   
17                   learning_rate   
18                       optimizer   
19                      num_epochs   
20                        patience   
21                      batch_size   

                                                value  
0                                              cuda:0  
1            

In [25]:
def zip_folder(folder_path, output_path):
    with zipfile.ZipFile(output_path, 'w', zipfile.ZIP_DEFLATED) as zipf:
        for root, dirs, files in os.walk(folder_path):
            for file in files:
                zipf.write(os.path.join(root, file))

folder_path = model_name
output_path = f'{model_name}.zip'

zip_folder(folder_path, output_path)